# code to increase contrast in the labels by assigning diffrent colors..

In [1]:
import os
import cv2
import numpy as np

def adjust_contrast(image, rgb_codes, new_values):
    """Adjust the contrast of an image based on new values for the RGB codes.
    
    Args:
        image (np.ndarray): Input image.
        rgb_codes (list): List of original RGB codes.
        new_values (list): List of new RGB values for better contrast.
        
    Returns:
        np.ndarray: Image with adjusted contrast.
    """
    # Create a mapping from original to new values
    mapping = {old: new for old, new in zip(rgb_codes, new_values)}
    
    # Create an output image
    output_image = np.zeros_like(image)
    
    # Apply the mapping
    for old, new in mapping.items():
        mask = (image == old).all(axis=-1)
        output_image[mask] = new
    
    return output_image

def process_folder(src_folder, dest_folder, rgb_codes, new_values):
    """Process all images in a folder to adjust their contrast and save them.
    
    Args:
        src_folder (str): Source folder containing the images.
        dest_folder (str): Destination folder to save the processed images.
        rgb_codes (list): List of original RGB codes.
        new_values (list): List of new RGB values for better contrast.
    """
    if not os.path.exists(dest_folder):
        os.makedirs(dest_folder)
    
    for filename in os.listdir(src_folder):
        if filename.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tif', '.tiff')):
            # Read the image
            src_path = os.path.join(src_folder, filename)
            image = cv2.imread(src_path)
            
            # Adjust the contrast
            adjusted_image = adjust_contrast(image, rgb_codes, new_values)
            
            # Save the adjusted image
            dest_path = os.path.join(dest_folder, filename)
            cv2.imwrite(dest_path, adjusted_image)

# Original RGB codes and new values for better contrast
rgb_codes = [(i, i, i) for i in range(7)]
new_values = [(int((255/7) * i), int((255/7) * i), int((255/7) * i)) for i in range(7)]

# Example usage for train folder
src_folder = r'D:\JIO Institute\Capstone\Capstone_V2\ID_code_UnetR\Multiclass-Image-Segmentation-using-UNETR-in-TensorFlow\casa_data_new\train\labels'
dest_folder = r'D:\JIO Institute\Capstone\Capstone_V2\ID_code_UnetR\Multiclass-Image-Segmentation-using-UNETR-in-TensorFlow\casa_data_new\train\labels_c'

process_folder(src_folder, dest_folder, rgb_codes, new_values)


In [2]:
# Example usage for the val folder
src_folder = r'D:\JIO Institute\Capstone\Capstone_V2\ID_code_UnetR\Multiclass-Image-Segmentation-using-UNETR-in-TensorFlow\casa_data_new\val\labels'
dest_folder = r'D:\JIO Institute\Capstone\Capstone_V2\ID_code_UnetR\Multiclass-Image-Segmentation-using-UNETR-in-TensorFlow\casa_data_new\val\labels_c'

process_folder(src_folder, dest_folder, rgb_codes, new_values)

## Code to Fit the model ..

In [15]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import numpy as np
import cv2
from glob import glob
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger, EarlyStopping


from tensorflow.keras.optimizers import SGD

# Suppress the specific warning about deprecated function
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
from tensorflow.keras.optimizers import Adam, SGD
from sklearn.model_selection import train_test_split
from patchify import patchify

# some_file.py
import sys
3
from unetr_2d import build_unetr_2d


""" UNETR  Configration """
cf = {}
cf["image_size"] = 256
cf["num_classes"] = 7
cf["num_channels"] = 3
cf["num_layers"] = 12
cf["hidden_dim"] = 128
cf["mlp_dim"] = 32
cf["num_heads"] = 6
cf["dropout_rate"] = 0.1
cf["patch_size"] = 16
cf["num_patches"] = (cf["image_size"]**2)//(cf["patch_size"]**2)
cf["flat_patches_shape"] = (
    cf["num_patches"],
    cf["patch_size"]*cf["patch_size"]*cf["num_channels"]
)

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def load_dataset(path):
    train_x = sorted(glob(os.path.join(path, "train", "images", "*.png")))
    train_y = sorted(glob(os.path.join(path, "train", "labels_c", "*.png")))

    valid_x = sorted(glob(os.path.join(path, "val", "images", "*.png")))
    valid_y = sorted(glob(os.path.join(path, "val", "labels_c", "*.png")))

    test_x = sorted(glob(os.path.join(path, "test", "images", "*.png")))
    test_y = sorted(glob(os.path.join(path, "test", "labels_c", "*.png")))

    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)

def read_image(path):
    path = path.decode()
    image = cv2.imread(path, cv2.IMREAD_COLOR)
    image = cv2.resize(image, (cf["image_size"], cf["image_size"]))
    image = image / 255.0

    """ Processing to patches """
    patch_shape = (cf["patch_size"], cf["patch_size"], cf["num_channels"])
    patches = patchify(image, patch_shape, cf["patch_size"])
    patches = np.reshape(patches, cf["flat_patches_shape"])
    patches = patches.astype(np.float32)

    return patches



################# change ##############
def read_mask(path):
    path = path.decode()
    mask = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    mask = cv2.resize(mask, (cf["image_size"], cf["image_size"]))
    mask = mask.astype(np.int32)
    return mask

def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        y = tf.one_hot(y, cf["num_classes"])
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape(cf["flat_patches_shape"])
    y.set_shape([cf["image_size"], cf["image_size"], cf["num_classes"]])
    return x, y

def tf_dataset(X, Y, batch=2):
    ds = tf.data.Dataset.from_tensor_slices((X, Y))
    ds = ds.map(tf_parse).batch(batch).prefetch(10)
    return ds

if __name__ == "__main__":
    """ Seeding """
    np.random.seed(42)
    tf.random.set_seed(42)

    """ Directory for storing files """
    create_dir("files")

    """ Hyperparameters """
    batch_size = 16
    lr = 0.1
    num_epochs = 100
    model_path = os.path.join(r"D:\JIO Institute\Capstone\Capstone_V2\ID_code_UnetR\Multiclass-Image-Segmentation-using-UNETR-in-TensorFlow\files", "model.h5")
    csv_path = os.path.join(r"D:\JIO Institute\Capstone\Capstone_V2\ID_code_UnetR\Multiclass-Image-Segmentation-using-UNETR-in-TensorFlow\files", "log.csv")

    """ RGB Code and Classes """
    rgb_codes = [
        (0, 0, 0), (36, 36, 36), (72, 72, 72), (109, 109, 109), (145, 145, 145), (182, 182, 182), (218, 218, 218)
    ]

    classes = [
        "background","head","acrosome","non-acrosome","midpiece","tail","noise"
    ]

    """ Dataset """
    dataset_path = r"D:\JIO Institute\Capstone\Capstone_V2\ID_code_UnetR\Multiclass-Image-Segmentation-using-UNETR-in-TensorFlow\casa_data_new"
    #(train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_dataset(dataset_path)
    (train_x, train_y), (valid_x, valid_y), (test_x, test_y)= load_dataset(dataset_path)

    print(f"Train: \t{len(train_x)} - {len(train_y)}")
    print(f"Valid: \t{len(valid_x)} - {len(valid_y)}")
    print(f"Test: \t{len(test_x)} - {len(test_y)}")


######### the error was in the load_data function where the type of images was set to jpg intead of png in our case....
    print(f"Shape of train_x: {len(train_x)}")
    print(f"Shape of train_y: {len(train_y)}")
    print(f"Shape of valid_x: {len(valid_x)}")
    print(f"Shape of valid_y: {len(valid_y)}")
    print(f"Shape of test_x: {len(test_x)}")
    print(f"Shape of test_y: {len(test_y)}")

################################################################################################################
###########################################   Model Training Begins  ###########################################
################################################################################################################
    train_dataset = tf_dataset(train_x, train_y, batch=batch_size)
    valid_dataset = tf_dataset(valid_x, valid_y, batch=batch_size)
    test_dataset = tf_dataset(test_x, test_y, batch=batch_size)

    """ Model """
   

    model = build_unetr_2d(cf)
    # Compiling the model with categorical crossentropy loss, SGD optimizer, and accuracy metric
    model.compile(loss="categorical_crossentropy", optimizer=SGD(learning_rate=lr), metrics=['accuracy'])


    

    callbacks = [
        ModelCheckpoint(model_path, verbose=1, save_best_only=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1),
        CSVLogger(csv_path),
        EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=False, verbose=1)
    ]

    model.fit(
        train_dataset,
        epochs=num_epochs,
        validation_data=valid_dataset,
        callbacks=callbacks
)

   


Train: 	122 - 122
Valid: 	137 - 137
Test: 	137 - 137
Shape of train_x: 122
Shape of train_y: 122
Shape of valid_x: 137
Shape of valid_y: 137
Shape of test_x: 137
Shape of test_y: 137
Epoch 1/100
8/8 [==============================] - ETA: 0s - loss: 1.3339 - accuracy: 0.6070
Epoch 1: val_loss improved from inf to 1.11673, saving model to D:\JIO Institute\Capstone\Capstone_V2\ID_code_UnetR\Multiclass-Image-Segmentation-using-UNETR-in-TensorFlow\files\model.h5


c:\Users\HamzaKhalid\envs\heart\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


8/8 [==============================] - 53s 5s/step - loss: 1.3339 - accuracy: 0.6070 - val_loss: 1.1167 - val_accuracy: 1.0000 - lr: 0.1000
Epoch 2/100
8/8 [==============================] - ETA: 0s - loss: 0.3755 - accuracy: 0.9976
Epoch 2: val_loss improved from 1.11673 to 0.70043, saving model to D:\JIO Institute\Capstone\Capstone_V2\ID_code_UnetR\Multiclass-Image-Segmentation-using-UNETR-in-TensorFlow\files\model.h5
8/8 [==============================] - 38s 5s/step - loss: 0.3755 - accuracy: 0.9976 - val_loss: 0.7004 - val_accuracy: 1.0000 - lr: 0.1000
Epoch 3/100
8/8 [==============================] - ETA: 0s - loss: 0.1626 - accuracy: 0.9999
Epoch 3: val_loss improved from 0.70043 to 0.49704, saving model to D:\JIO Institute\Capstone\Capstone_V2\ID_code_UnetR\Multiclass-Image-Segmentation-using-UNETR-in-TensorFlow\files\model.h5
8/8 [==============================] - 36s 5s/step - loss: 0.1626 - accuracy: 0.9999 - val_loss: 0.4970 - val_accuracy: 1.0000 - lr: 0.1000
Epoch 4/100


# Code to fit model using cuda on local device:

In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import numpy as np
import cv2
from glob import glob
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.optimizers import SGD
from sklearn.model_selection import train_test_split
from patchify import patchify

# some_file.py
import sys

from unetr_2d import build_unetr_2d

""" UNETR  Configration """
cf = {}
cf["image_size"] = 256
cf["num_classes"] = 7
cf["num_channels"] = 3
cf["num_layers"] = 12
cf["hidden_dim"] = 128
cf["mlp_dim"] = 32
cf["num_heads"] = 6
cf["dropout_rate"] = 0.1
cf["patch_size"] = 16
cf["num_patches"] = (cf["image_size"]**2)//(cf["patch_size"]**2)
cf["flat_patches_shape"] = (
    cf["num_patches"],
    cf["patch_size"]*cf["patch_size"]*cf["num_channels"]
)

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def load_dataset(path):
    train_x = sorted(glob(os.path.join(path, "train", "images", "*.png")))
    train_y = sorted(glob(os.path.join(path, "train", "labels_c", "*.png")))

    valid_x = sorted(glob(os.path.join(path, "val", "images", "*.png")))
    valid_y = sorted(glob(os.path.join(path, "val", "labels_c", "*.png")))
    
    test_x = sorted(glob(os.path.join(path, "test", "images", "*.png")))
    test_y = sorted(glob(os.path.join(path, "test", "labels_c", "*.png")))

    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)

def read_image(path):
    path = path.decode()
    image = cv2.imread(path, cv2.IMREAD_COLOR)
    image = cv2.resize(image, (cf["image_size"], cf["image_size"]))
    image = image / 255.0

    """ Processing to patches """
    patch_shape = (cf["patch_size"], cf["patch_size"], cf["num_channels"])
    patches = patchify(image, patch_shape, cf["patch_size"])
    patches = np.reshape(patches, cf["flat_patches_shape"])
    patches = patches.astype(np.float32)

    return patches

def read_mask(path):
    path = path.decode()
    mask = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    mask = cv2.resize(mask, (cf["image_size"], cf["image_size"]))
    mask = mask.astype(np.int32)
    return mask

def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        y = tf.one_hot(y, cf["num_classes"])
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape(cf["flat_patches_shape"])
    y.set_shape([cf["image_size"], cf["image_size"], cf["num_classes"]])
    return x, y

def tf_dataset(X, Y, batch=2):
    ds = tf.data.Dataset.from_tensor_slices((X, Y))
    ds = ds.map(tf_parse).batch(batch).prefetch(10)
    return ds

if __name__ == "__main__":
    """ Seeding """
    np.random.seed(42)
    tf.random.set_seed(42)

    """ Directory for storing files """
    # create_dir("files")

    """ Hyperparameters """
    batch_size = 16
    lr = 0.1
    num_epochs = 100
    model_path = os.path.join(r"D:\JIO Institute\Capstone\Capstone_V2\ID_code_UnetR\Multiclass-Image-Segmentation-using-UNETR-in-TensorFlow\files", "model.h5")
    csv_path = os.path.join(r"D:\JIO Institute\Capstone\Capstone_V2\ID_code_UnetR\Multiclass-Image-Segmentation-using-UNETR-in-TensorFlow\files", "log.csv")

    """ RGB Code and Classes """
    rgb_codes = [
        (0, 0, 0), (36, 36, 36), (72, 72, 72), (109, 109, 109), (145, 145, 145), (182, 182, 182), (218, 218, 218)
    ]

    classes = [
        "background","head","acrosome","non-acrosome","midpiece","tail","noise"
    ]

    """ Dataset """
    dataset_path = r"D:\JIO Institute\Capstone\Capstone_V2\ID_code_UnetR\Multiclass-Image-Segmentation-using-UNETR-in-TensorFlow\casa_data_new"
    (train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_dataset(dataset_path)



    train_dataset = tf_dataset(train_x, train_y, batch=batch_size)
    valid_dataset = tf_dataset(valid_x, valid_y, batch=batch_size)
    test_dataset = tf_dataset(test_x, test_y, batch=batch_size)

    """ Model """
    model = build_unetr_2d(cf)
    model.compile(loss="categorical_crossentropy", optimizer=SGD(lr))

    callbacks = [
        ModelCheckpoint(model_path, verbose=1, save_best_only=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1),
        CSVLogger(csv_path),
        EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=False)
    ]

    with tf.device('/device:GPU:0'):
        model.fit(
            train_dataset,
            epochs=num_epochs,
            validation_data=valid_dataset,
            callbacks=callbacks
        )


## Testing our model's output :

In [21]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import numpy as np
import cv2
import pandas as pd
from glob import glob
from tqdm import tqdm
import tensorflow as tf
from patchify import patchify


from unetr_2d import build_unetr_2d

from train import load_dataset, create_dir

""" UNETR  Configration """
cf = {}
cf["image_size"] = 256
cf["num_classes"] = 7
cf["num_channels"] = 3
cf["num_layers"] = 12
cf["hidden_dim"] = 128
cf["mlp_dim"] = 32
cf["num_heads"] = 6
cf["dropout_rate"] = 0.1
cf["patch_size"] = 16
cf["num_patches"] = (cf["image_size"]**2)//(cf["patch_size"]**2)
cf["flat_patches_shape"] = (
    cf["num_patches"],
    cf["patch_size"]*cf["patch_size"]*cf["num_channels"]
)

def grayscale_to_rgb(mask, rgb_codes):
    h, w = mask.shape[0], mask.shape[1]
    mask = mask.astype(np.int32)
    output = []

    for i, pixel in enumerate(mask.flatten()):
        output.append(rgb_codes[pixel])

    output = np.reshape(output, (h, w, 3))
    return output


def load_dataset(path):
    train_x = sorted(glob(os.path.join(path, "train", "images", "*.png")))
    train_y = sorted(glob(os.path.join(path, "train", "labels_c", "*.png")))

    valid_x = sorted(glob(os.path.join(path, "val", "images", "*.png")))
    valid_y = sorted(glob(os.path.join(path, "val", "labels_c", "*.png")))

    test_x = sorted(glob(os.path.join(path, "test", "images", "*.png")))
    test_y = sorted(glob(os.path.join(path, "test", "labels_c", "*.png")))

    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)


def save_results(image_x, mask, pred, save_image_path):
    mask = np.expand_dims(mask, axis=-1)
    mask = grayscale_to_rgb(mask, rgb_codes)

    pred = np.expand_dims(pred, axis=-1)
    pred = grayscale_to_rgb(pred, rgb_codes)

    line = np.ones((image_x.shape[0], 10, 3)) * 255

    cat_images = np.concatenate([image_x, line, mask, line, pred], axis=1)
    cv2.imwrite(save_image_path, cat_images)



if __name__ == "__main__":
    """ Seeding """
    np.random.seed(42)
    tf.random.set_seed(42)

    """ Directory for storing files """
    create_dir(f"results")

    """ Load the model """
    model_path = os.path.join(r"D:\JIO Institute\Capstone\Capstone_V2\ID_code_UnetR\Multiclass-Image-Segmentation-using-UNETR-in-TensorFlow\files", "model.h5")
    model = tf.keras.models.load_model(model_path)

    """ RGB Code and Classes """
    rgb_codes = [
        (0, 0, 0), (36, 36, 36), (72, 72, 72), (109, 109, 109), (145, 145, 145), (182, 182, 182), (218, 218, 218)
    ]

    classes = [
        "background", "head" ,"acrosome" ,"non-acrosome" ,"midpiece" ,"tail" ,"noise"
    ]

    """ Dataset """
    dataset_path = r"D:\JIO Institute\Capstone\Capstone_V2\ID_code_UnetR\Multiclass-Image-Segmentation-using-UNETR-in-TensorFlow\casa_data_new"
    (train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_dataset(dataset_path)
    

    ## as we donot currently have a test data directory .... we're simply using the valid directory as the test directory !!


    """ Prediction """
    
    ######################################################### new code that  multiplies pred by 225 #################################################################

    # for x, y in tqdm(zip(test_x, test_y), total=len(test_x)):
    #     """ Extracting the name """
    #     name = os.path.basename(x).split(".")[0]

    #     """ Reading the image """
    #     image = cv2.imread(x, cv2.IMREAD_COLOR)
    #     image = cv2.resize(image, (cf["image_size"], cf["image_size"]))
    #     x = image / 255.0

    #     patch_shape = (cf["patch_size"], cf["patch_size"], cf["num_channels"])
    #     patches = patchify(x, patch_shape, cf["patch_size"])
    #     patches = np.reshape(patches, cf["flat_patches_shape"])
    #     patches = patches.astype(np.float32) #[...]
    #     patches = np.expand_dims(patches, axis=0) # [1, ...]

    #     """ Read Mask """
    #     mask = cv2.imread(y, cv2.IMREAD_GRAYSCALE)
    #     mask = cv2.resize(mask, (cf["image_size"], cf["image_size"]))
    #     mask = mask.astype(np.int32)

    #     """ Prediction """
    #     pred = model.predict(patches, verbose=0)[0]
    #     pred = np.argmax(pred, axis=-1) ## [0.1, 0.2, 0.1, 0.6] -> 3
    #     pred = pred.astype(np.int32)

    #     """ Scale the predicted values back to 0-255 """
    #     pred = pred * 255
    #     pred = np.clip(pred, 0, 255)  # Ensure the values are within 0-255
    #     pred = pred.astype(np.uint8)

    #     """ Save the results """
    #     save_image_path = os.path.join(r"D:\JIO Institute\Capstone\Capstone_V2\ID_code_UnetR\Multiclass-Image-Segmentation-using-UNETR-in-TensorFlow\results",f"{name}.png")
    #     save_results(image, mask, pred, save_image_path)
    
    
    ######################################################### old code that wasn't multiplying by 225 #################################################################
    
    for x, y in tqdm(zip(test_x, test_y), total=len(test_x)):
        """ Extracting the name """
        name = x.split("/")[-1].split(".")[0]

        """ Reading the image """
        image = cv2.imread(x, cv2.IMREAD_COLOR)
        image = cv2.resize(image, (cf["image_size"], cf["image_size"]))
        x = image / 255.0

        patch_shape = (cf["patch_size"], cf["patch_size"], cf["num_channels"])
        patches = patchify(x, patch_shape, cf["patch_size"])
        patches = np.reshape(patches, cf["flat_patches_shape"])
        patches = patches.astype(np.float32) #[...]
        patches = np.expand_dims(patches, axis=0) # [1, ...]

        """ Read Mask """
        mask = cv2.imread(y, cv2.IMREAD_GRAYSCALE)
        mask = cv2.resize(mask, (cf["image_size"], cf["image_size"]))
        mask = mask.astype(np.int32)

        """ Prediction """
        pred = model.predict(patches, verbose=0)[0]
        pred = np.argmax(pred, axis=-1) ## [0.1, 0.2, 0.1, 0.6] -> 3
        pred = pred.astype(np.int32)

        """ Save the results """
        save_image_path = os.path.join(r"D:\JIO Institute\Capstone\Capstone_V2\ID_code_UnetR\Multiclass-Image-Segmentation-using-UNETR-in-TensorFlow\results",f"{name}.png")
        save_results(image, mask, pred, save_image_path)
    
    

  0%|          | 0/137 [00:01<?, ?it/s]


IndexError: list index out of range